POWERMAX_Tonka
Notebook created 03/30/23 from OptionGun03-Tonka-Prdtn
Last rev 06/14/23 - Corrected some run-time errors in the docker version. Runs in docker!
Uses full OptionGunLib01.py and full PowerMax01 Main

User Notes
1. This program needs a user key from Darqube.com, which it uses to get real-time stock prices.
   Go to the Darqube website get a user account and choose the free service. Then get your key.
   Find the Dar_key line just below the "User Parameters" block in the cell below.
   Enter it between the quotes on the line:  DAR_key = ''
   
2. OptionGun produces these csv files:
     all_options_suffix.csv. This file contains every option available for every ticker in the list. 
     Put_STO_short_Tonka.csv. Best STO options that meet criteria and expiry < 50 days out.
     Put_STO_mid_Tonka.csv.   Best STO options that meet criteria and 50 days <= expiry < 150 days out.
     Put_STO_long_Tonka.csv   Best STO options that meet criteria and 150 days out <= expiry.
     
     Call_STO_short_Tonka.csv. Best STO options that meet criteria and expiry < 50 days out.
     Call_STO_mid_Tonka.csv.   Best STO options that meet criteria and 50 days <= expiry < 150 days out.
     Call_STO_long_Tonka.csv   Best STO options that meet criteria and 150 days out <= expiry.
     
     root_data.csv. This contains a list of all tickers, with real-time price at the time of execution,
     and the netx Earnings Report date for each ticker, if available. DO NOT CHANGE THIS FILE.

2. The parameters just under the "User Parameters" header block can be changed.
    Add and remove ticker symbols from the TICKERS = list. Enclose each ticker Follow the format shown.
    HIDE_TICKERS. You may want to collect option info for a ticker, but not see that ticker in best_file.csv.
    Just put the ticker to be hidden in this list.
    FILE_ID_TAG is the suffix that OptionGun will attach to all_options and best_options. You can change it at any time.
    
3. PowerMax needs about 2-8 seconds to get the option string for each ticker. For 200 tickers, it will take about 25 minutes.



In [1]:
# ****************
# USER ADJUSTABLE PARAMETERS
# *****************

DAR_key = ''

TICKERS = ('AAPL', 'ABBV', 'ABT', 'ACLS', 'ACRE', 'AFL', 'ADBE', 'AFYA', 'AIG', 'AMD', 'AMGN', 'AMP', 'AMR', 'AMT', 'AMZN', \
           'AOS', 'ARCC', 'ARES', 'ARNC', 'ASC', 'ASRT', 'ASO', 'AVB', 'AVGO', 'AX', 'AZO', 'BA', 'BAC', 'BGSF', 'BIIB', 'BNTX', 'BRSP', \
           'BSM', 'CAG', 'CAT', 'CCI', 'CEG', 'CF', 'CFR', 'CL', 'CLX', 'CMCSA', 'CMG', 'CPA', 'CPT', 'CRM', 'CRWD', 'CSCO', 'CSX', \
           'CUBE', 'CVCO', 'CVS', 'CVX', 'DAC', 'DDOG', 'DHI', 'DLR', 'DOCU', 'DUK', 'DVN', 'EPAM', 'EPD', 'EPR', 'ET', 'EXC', 'EXPE', \
           'EXR', 'FAF', 'FANG', 'FCNCA', 'FDX', 'FITB', 'FLNG', 'FMS', 'FNF', 'FSLY', 'FSR', 'FYBR', 'GBX', 'GD', 'GIS', 'GLNG', \
           'GOOGL', 'GS', 'GSL', 'HD', 'HDSN', 'HIW', 'HON', 'HR', 'HRB', 'HUBG', 'IBM', 'INTC', 'IPI', 'IRM', 'JEF', 'JNJ', 'JPM', \
           'JXN', 'K', 'KEY', 'KO', 'LADR', 'LLY', 'LMT', 'LOW', 'LRCX', 'LYB', 'MA', 'MAN', 'MCK', 'MDB', 'META', 'MGA', 'MLM', 'MMM', 'MMP', \
           'MNSO', 'MOD', 'MOS', 'MPW', 'MRK', 'MRNA', 'MRVL', 'MS', 'MSFT', 'MTB', 'MTCH', 'MU', 'NFLX', 'NKE', 'NKTR', 'NMM', 'NOW', \
           'NRG', 'NUE', 'NVDA', 'NWG', 'NYCB', 'OKTA', 'ON', 'ORCC', 'PAA', 'PACW', 'PANW', 'PB', 'PD', 'PERI', 'PFE', 'PFG', 'PGR', 'PHM', \
           'PII', 'PINE', 'PLTR', 'PNC', 'PRU', 'PSA', 'PYPL', 'QCOM', 'RC', 'RILY', 'RITM', 'RIVN', 'ROST', 'RTX', 'SB', 'SBLK', 'SBUX', 'SCHW', \
           'SEDG', 'SKT', 'SMCI', 'SNOW', 'SO', 'SPG', 'SPY', 'SRC', 'STAG', 'STLD', 'STRL', 'STWD', 'TMUS', 'TSLA', 'TRV', 'TXN', 'TSCO', \
           'TSN', 'UBS', 'UHAL', 'UNP', 'UNVR', 'UPS', 'URI', 'USAC', 'USB', 'V', 'VICI', 'VIR', 'VGT', 'VIST', 'VLO', 'VMW', 'VNOM', 'VOO', \
           'VPG', 'VRNA', 'WDC', 'WEC', 'WMT', 'WSM', 'WFC', 'XPEL', 'XOM', 'ZIM', 'ZS')

HIDE_TICKERS = ('AOS', 'FSLY', 'NOW', 'PLTR', 'TSLA', 'VNOM', 'RIVN', 'NKTR') # Keep these tickers for database, but do not show them in best_options, because you do not want to own them.  

#Short list for testing
#TICKERS = ('QCOM', 'GOOGL', 'META', 'NVDA')


In [2]:
# ****************
# PowerMax01 - 04/02/23.
# *****************
# Goal - PowerMax runs all four main functional blocks: TradeManager, ScoreKeeper, CrystalBall, and OptionGun. 
#
# 
#   ToDo - Move GetERDates to CrystalBall. Not part of OptionGun.
#          Delete extra columns from saved STO profiles. What went wrong with existing delete? Reconcile puts_all with Putsssa
#
#   04/04/23 - Runs, with warnings in Bullets_PSTO. Fix warnings.
#   04/02/23 - Created from OptionGun03-Bullets.
#
# *****************
#
# Import Libraries
#
# *****************

import csv
import copy
import json
import numpy as np
import os
import pandas as pd
import requests   # for http requests
import scipy
from   scipy import stats
from   scipy.stats import norm
import time
import yfinance as yf

import datetime
from   datetime import datetime
from   datetime import date
from   datetime import timedelta
#from   datetime import fromtimestamp

from   dateutil.relativedelta  import relativedelta
#from   yahoo_earnings_calendar import YahooEarningsCalendar
# replace this with a better earnings calendar.
import OptionGunLib01

#from OptionGunLib01 import GetERDates
from OptionGunLib01 import GetPrices
from OptionGunLib01 import GetOptions
from OptionGunLib01 import BuildOptionMetrics
from OptionGunLib01 import Bullets_STO


# *****************
#
# Flow Control
#
# CHECK 3 PARAMS:  TICKERS, FILE_ID_TAG, profile
#
# *****************

update_root_data       = True  #skip Darqube call for tests.
update_root_data_ER    = False  #skip call to yahoo calendar for tests. 
update_option_data     = True  #skip call to yfinance if testing code.
run_BuildOptionMetrics = update_option_data  #skip call to BuildOptionMetrics if no update of option data.


# *****************
#
# CONSTANTS
#
# *****************


print(len(TICKERS), ' tickers to analyze.')
FILE_ID_TAG       = 'Tonka'
DATE_TIME_FORMAT  = '%Y-%m-%d %H:%M:%S'
DATE_ONLY_FORMAT  = '%Y-%m-%d'

TICKER_DELAY  = 0.015  # seconds between tickers for data requests.
EXPIRY_DELAY  = 0.025
YEC_DELAY     = 0.75

MAX_DAYSOUT   = 500.0  # This constant controls how many expiry strings are downloaded. Use low # to exclude BTOS and reduce ext. calls. Hi # includes BTOS.
MIN_DAYSOUT   = -1.0   

FEE_SPREAD    = 0.75     # Fee in the bid/ask range.
DAYS_TO_CHECK_ER = (1,4) #Update ER dates from yahoo on these days of the month.

call_ticker_options = pd.DataFrame()
put_ticker_options  = pd.DataFrame()
ticker_options      = pd.DataFrame() # for a single ticker
all_options         = pd.DataFrame() # includes options for all tickers. Do not use any limits or exclusions for all_options.
good_options        = pd.DataFrame() # Has rankings for CSTO options and BTOs (CBTO).
stats               = pd.DataFrame() # Not used now.
root_data           = pd.DataFrame() # persistent file used by other programs. Specifically, OptionJrnlAll spreadsheet. This programu updates it.
options_open        = pd.DataFrame() # persistent file used by OptionCloser to recommend BTC offers.

# *****************
#
# MAIN PROGAM STARTS HERE (next 2 cells).
# 1. If stock market closed and broker_report file in folder
#       run TradeLogManager code block in Main program.'
#       run ScoreKeeper code block in Main program.
# 2. If stock market closed run CrystalBall code block in Main program.
# 3, If stock market open run OptionGun code block in Main program.
#
# *****************


OptionGunLib01 runs like a top.
217  tickers to analyze.


In [3]:
# *****************
#
# TradeManager
#
# 1. If stock market closed and broker_report file in folder
#       run TradeLogManager code block in Main program.'
# 2. Run BrokerReporter if new broker_report file is in folder.
# 3. Run TradeLogger, which updates trades_open and trades_closed
# 4. Update holdings, cost basis, # contracts committed, # contracts available. All these data columns are in root_data.

# *****************
#
# ScoreKeeper
#
# 1. Run sll ScoreKeeper functions.
#
# *****************

# *****************
#
# CrystalBall
#
# 1. Update all ML features for all option profiles.
# 2. Run ScoreKeeper code block in Main program.
# 2. If stock market closed run CrystalBall code block in Main program.
# 3, If stock market open run OptionGun code block in Main program.
#
# *****************

In [4]:
# *****************
#
#OptionGun
#
# 1. Call GetPrices to get real-time price data from Darqube write to root_data. 
# 2. Call GetERDates to get dates from Yahoo Earnings Calendar and write to root_data.
# 3. Call GetOptions to get options data from yfinance. Use "try" statements to skip over exp dates where call to yfinance failed.
# 4. Call BuildOptionMetrics to build additional metrics fields in each option record.
# 5. Call Bullets_STO to screen all_options for each STO profile.
# 6. Call Bullets_BTC to build BTC trades for each open STO trade.
# 7. Call BuyWrite to screen for BuyWrite trades.
# 
# *****************

root_data = pd.read_csv('root_data.csv')   #Get root_data price and ER info.
root_data.set_index('Ticker', inplace = True, drop = True)
print('just read root_data')
#Use Darqube for stock price data - both market and historical.

print('start Darqube access at:  ', datetime.now(), '  Done in ',len(TICKERS) * 1.0 / 200.0, ' minutes.')

if update_root_data:
    root_data = GetPrices(TICKERS, root_data, DAR_key)

print(datetime.now())

update_root_data_ER = False            # ToDo - Find a good ER calendar. YEC does not work.
if update_root_data_ER:                # skip call to YEC for ER update
    if date.today().day in DAYS_TO_CHECK_ER:
        print('Updating ER dates.')
        root_data = GetERDates(root_data, TICKERS, YEC_DELAY)

root_data.to_csv('root_data.csv')

# Get option chains
if update_option_data:
    all_options = GetOptions(root_data, TICKERS, EXPIRY_DELAY, TICKER_DELAY, MAX_DAYSOUT, MIN_DAYSOUT)
else:
    all_options = pd.read_csv('all_options23-04-03-14.csv', index_col = 0)
    
#Calculate additional columns for all_options
if run_BuildOptionMetrics:
    all_options, clean_options = BuildOptionMetrics(all_options, FEE_SPREAD, HIDE_TICKERS, DATE_TIME_FORMAT)
    write_file = 'all_options' + FILE_ID_TAG + '.csv'
    all_options.to_csv(write_file)
    print('Wrote all_options')
else:
    all_options = pd.read_csv('all_options23-03-24-12.csv') # For testing only    


profiles = pd.read_csv('option_profiles.csv', index_col = 0)
float_columns = profiles.columns.tolist()
float_columns.remove('Active') # So, why have 'Active'? Maybe for later testing and run control?.
profiles[float_columns] = profiles[float_columns].astype(float).copy()

print('calling Bullets_STO')
Bullets_STO(clean_options, profiles)
print('Bullets_STO ran all the way.')


just read root_data
start Darqube access at:   2023-06-15 10:16:51.196067   Done in  1.085  minutes.
DAR_key =  90180f15ecc74513a01ca017eca2bb4f    TICKERS[0] =  AAPL
getting price for  AAPL
getting price for  ABBV
getting price for  ABT
getting price for  ACLS
getting price for  ACRE
getting price for  AFL
getting price for  ADBE
getting price for  AFYA
getting price for  AIG
getting price for  AMD
getting price for  AMGN
getting price for  AMP
getting price for  AMR
getting price for  AMT
getting price for  AMZN
getting price for  AOS
getting price for  ARCC
getting price for  ARES
getting price for  ARNC
getting price for  ASC
getting price for  ASRT
getting price for  ASO
getting price for  AVB
getting price for  AVGO
getting price for  AX
getting price for  AZO
getting price for  BA
getting price for  BAC
getting price for  BGSF
getting price for  BIIB
getting price for  BNTX
getting price for  BRSP
getting price for  BSM
getting price for  CAG
getting price for  CAT
getting price

getting data for  CVCO  at time  2023-06-15 10:21:08.458355
getting data for  CVS  at time  2023-06-15 10:21:09.927399
getting data for  CVX  at time  2023-06-15 10:21:14.112395
getting data for  DAC  at time  2023-06-15 10:21:19.003657
getting data for  DDOG  at time  2023-06-15 10:21:20.519627
getting data for  DHI  at time  2023-06-15 10:21:25.451608
getting data for  DLR  at time  2023-06-15 10:21:29.529054
getting data for  DOCU  at time  2023-06-15 10:21:33.379000
getting data for  DUK  at time  2023-06-15 10:21:38.171448
getting data for  DVN  at time  2023-06-15 10:21:40.081607
getting data for  EPAM  at time  2023-06-15 10:21:45.125114
getting data for  EPD  at time  2023-06-15 10:21:46.866326
getting data for  EPR  at time  2023-06-15 10:21:50.676433
getting data for  ET  at time  2023-06-15 10:21:52.304598
getting data for  EXC  at time  2023-06-15 10:21:55.784711
getting data for  EXPE  at time  2023-06-15 10:21:57.613956
getting data for  EXR  at time  2023-06-15 10:22:01.

getting data for  UBS  at time  2023-06-15 10:28:49.781081
getting data for  UHAL  at time  2023-06-15 10:28:51.645861
getting data for  UNP  at time  2023-06-15 10:28:53.180520
getting data for  UNVR  at time  2023-06-15 10:28:57.323620
getting data for  UPS  at time  2023-06-15 10:28:59.217914
getting data for  URI  at time  2023-06-15 10:29:03.395207
getting data for  USAC  at time  2023-06-15 10:29:07.751341
getting data for  USB  at time  2023-06-15 10:29:09.215433
getting data for  V  at time  2023-06-15 10:29:13.651208
getting data for  VICI  at time  2023-06-15 10:29:18.245705
getting data for  VIR  at time  2023-06-15 10:29:20.102917
getting data for  VGT  at time  2023-06-15 10:29:22.238965
getting data for  VIST  at time  2023-06-15 10:29:24.659060
getting data for  VLO  at time  2023-06-15 10:29:26.210820
getting data for  VMW  at time  2023-06-15 10:29:30.370025
getting data for  VNOM  at time  2023-06-15 10:29:32.732226
getting data for  VOO  at time  2023-06-15 10:29:34.

C:\Users\richa\OneDrive\Documents\GitHub\POWMax_Tonka_Prdtn\OptionGunLib01.py:249: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  best_options = best_options.merge(ticker_options, how = 'outer', copy = True)


Wrote  Put_STO_Mid  at  2023-06-15 10:30:26.876281


...

Wrote  Call_STO_Short  at  2023-06-15 10:30:38.169505


C:\Users\richa\OneDrive\Documents\GitHub\POWMax_Tonka_Prdtn\OptionGunLib01.py:249: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  best_options = best_options.merge(ticker_options, how = 'outer', copy = True)


Wrote  Call_STO_Mid  at  2023-06-15 10:30:41.727456


C:\Users\richa\OneDrive\Documents\GitHub\POWMax_Tonka_Prdtn\OptionGunLib01.py:249: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  best_options = best_options.merge(ticker_options, how = 'outer', copy = True)
C:\Users\richa\OneDrive\Documents\GitHub\POWMax_Tonka_Prdtn\OptionGunLib01.py:249: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  best_options = best_options.merge(ticker_options, how = 'outer', copy = True)


Wrote  Call_STO_Long  at  2023-06-15 10:30:54.727759
Bullets_STO ran all the way.
